In [1]:
import xarray as xr
from FINE import utils
import FINE as fn
import numpy as np
import pandas as pd
import os
import warnings

import geopandas as gpd

In [3]:
def getData():
    inputDataPath = r'C:\Users\s.patil\Documents\code\fine\examples\Multi-regional_Energy_System_Workflow\InputData'
    data = {}

    # Onshore data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Wind', 'maxCapacityOnshore_GW_el.xlsx'),
                                index_col=0, squeeze=True)
    operationRateMax = pd.read_excel(
        os.path.join(inputDataPath, 'SpatialData', 'Wind', 'maxOperationRateOnshore_el.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Wind (onshore), capacityMax': capacityMax})
    data.update({'Wind (onshore), operationRateMax': operationRateMax})

    # Offshore data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Wind', 'maxCapacityOffshore_GW_el.xlsx'),
                                index_col=0, squeeze=True)
    operationRateMax = pd.read_excel(
        os.path.join(inputDataPath, 'SpatialData', 'Wind', 'maxOperationRateOffshore_el.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Wind (offshore), capacityMax': capacityMax})
    data.update({'Wind (offshore), operationRateMax': operationRateMax})

    # PV data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'PV', 'maxCapacityPV_GW_el.xlsx'),
                                index_col=0, squeeze=True)
    operationRateMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'PV', 'maxOperationRatePV_el.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'PV, capacityMax': capacityMax})
    data.update({'PV, operationRateMax': operationRateMax})

    # Run of river data
    capacityFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'HydroPower', 'fixCapacityROR_GW_el.xlsx'),
                                index_col=0, squeeze=True)
    operationRateFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'HydroPower',
                                                  'fixOperationRateROR.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Existing run-of-river plants, capacityFix': capacityFix})
    data.update({'Existing run-of-river plants, operationRateFix': operationRateFix})

    # Biogas data
    operationRateMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Biogas',
                                                  'biogasPotential_GWh_biogas.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Biogas, operationRateMax': operationRateMax})

    biogasCommodityCostTimeSeries = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Biogas',
                                                  'biogasPriceTimeSeries_MrdEuro_GWh.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Biogas, commodityCostTimeSeries': biogasCommodityCostTimeSeries})

    # Natural gas data
    naturalGasCommodityCostTimeSeries = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'NaturalGas',
                                                  'naturalGasPriceTimeSeries_MrdEuro_GWh.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Natural Gas, commodityCostTimeSeries': naturalGasCommodityCostTimeSeries})

    # Natural gas plant data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'NaturalGasPlants',
                                             'existingCombinedCycleGasTurbinePlantsCapacity_GW_el.xlsx'),
                                index_col=0, squeeze=True)

    data.update({'Existing CCGT plants (methane), capacityMax': capacityMax})

    # Hydrogen salt cavern data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'GeologicalStorage',
                                             'existingSaltCavernsCapacity_GWh_methane.xlsx'),
                                index_col=0, squeeze=True) * 3 / 10

    data.update({'Salt caverns (hydrogen), capacityMax': capacityMax})

    # Methane salt cavern data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'GeologicalStorage',
                                             'existingSaltCavernsCapacity_GWh_methane.xlsx'),
                                index_col=0, squeeze=True)

    data.update({'Salt caverns (methane), capacityMax': capacityMax})

    # Pumped hydro storage data
    capacityFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'HydroPower',
                                             'fixCapacityPHS_storage_GWh_energyPHS.xlsx'),
                                index_col=0, squeeze=True)

    data.update({'Pumped hydro storage, capacityFix': capacityFix})

    # AC cables data
    capacityFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                             'ACcableExistingCapacity_GW_el.xlsx'),
                                index_col=0, header=0)

    data.update({'AC cables, capacityFix': capacityFix})

    reactances = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                            'ACcableReactance.xlsx'),
                                index_col=0, header=0)

    data.update({'AC cables, reactances': reactances})

    # DC cables data
    capacityFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                             'DCcableExistingCapacity_GW_el.xlsx'),
                                index_col=0, header=0)
    distances = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                           'DCcableLength_km.xlsx'),
                              index_col=0, header=0)
    losses = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                        'DCcableLosses.xlsx'),
                           index_col=0, header=0)

    data.update({'DC cables, capacityFix': capacityFix})
    data.update({'DC cables, distances': distances})
    data.update({'DC cables, losses': losses})

    # Pipelines data
    eligibility = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Pipelines',
                                             'pipelineIncidence.xlsx'), index_col=0, header=0)
    distances = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Pipelines', 'pipelineLength.xlsx'),
                              index_col=0, header=0)

    data.update({'Pipelines, eligibility': eligibility})
    data.update({'Pipelines, distances': distances})

    # Electricity demand data
    operationRateFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Demands',
                                                  'electricityDemand_GWh_el.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Electricity demand, operationRateFix': operationRateFix})

    # Hydrogen demand data
    operationRateFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Demands',
                                                  'hydrogenDemand_GWh_hydrogen.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Hydrogen demand, operationRateFix': operationRateFix})

    return data  

data = getData()  # 2. Create an energy system model instance
locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
commodityUnitDict = {'electricity': r'GW$_{el}$', 'methane': r'GW$_{CH_{4},LHV}$', 'biogas': r'GW$_{biogas,LHV}$',
                         'CO2': r'Mio. t$_{CO_2}$/h', 'hydrogen': r'GW$_{H_{2},LHV}$'}
commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                               commodityUnitsDict=commodityUnitDict,
                               hoursPerTimeStep=1, costUnit='1e9 Euro', lengthUnit='km', verboseLogLevel=0)

CO2_reductionTarget = 1


# 3. Add commodity sources to the energy system model
## 3.1. Electricity sources
### Wind onshore

esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (onshore), operationRateMax'],
                  capacityMax=data['Wind (onshore), capacityMax'],
                  investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                  economicLifetime=20))

data['Wind (onshore), operationRateMax'].sum()


### Wind offshore

esM.add(fn.Source(esM=esM, name='Wind (offshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (offshore), operationRateMax'],
                  capacityMax=data['Wind (offshore), capacityMax'],
                  investPerCapacity=2.3, opexPerCapacity=2.3*0.02, interestRate=0.08,
                  economicLifetime=20))

data['Wind (offshore), operationRateMax'].sum()

### PV

esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))

data['PV, operationRateMax'].sum()

### Exisisting run-of-river hydroelectricity plants

esM.add(fn.Source(esM=esM, name='Existing run-of-river plants', commodity='electricity',
                  hasCapacityVariable=True,
                  operationRateFix=data['Existing run-of-river plants, operationRateFix'], tsaWeight=0.01,
                  capacityFix=data['Existing run-of-river plants, capacityFix'],
                  investPerCapacity=0, opexPerCapacity=0.208))

## 3.2. Methane (natural gas and biogas)
### Natural gas
esM.add(fn.Source(esM=esM, name='Natural gas purchase', commodity='methane',
                  hasCapacityVariable=False, commodityCostTimeSeries=data['Natural Gas, commodityCostTimeSeries']))

### Biogas
esM.add(fn.Source(esM=esM, name='Biogas purchase', commodity='biogas',
                  operationRateMax=data['Biogas, operationRateMax'], hasCapacityVariable=False,
                  commodityCostTimeSeries=data['Biogas, commodityCostTimeSeries']))


## 3.3 CO2
### CO2

esM.add(fn.Source(esM=esM, name='CO2 from enviroment', commodity='CO2',
                  hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))


# 4. Add conversion components to the energy system model

### Combined cycle gas turbine plants

esM.add(fn.Conversion(esM=esM, name='CCGT plants (methane)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625},
                      hasCapacityVariable=True,
                      investPerCapacity=0.65, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))


### New combined cycle gas turbine plants for biogas

esM.add(fn.Conversion(esM=esM, name='New CCGT plants (biogas)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'biogas':-1/0.635},
                      hasCapacityVariable=True,
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))


### New combined cycly gas turbines for hydrogen

esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True,
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### Electrolyzers

esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True,
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

### rSOC

capexRSOC=1.5

esM.add(fn.Conversion(esM=esM, name='rSOEC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.6},
                      hasCapacityVariable=True,
                      investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
                      economicLifetime=10))

esM.add(fn.Conversion(esM=esM, name='rSOFC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True,
                      investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
                      economicLifetime=10))


# 5. Add commodity storages to the energy system model
## 5.1. Electricity storage
### Lithium ion batteries

esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1, doPreciseTsaModeling=False,
                   investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                   economicLifetime=22))

## 5.2. Hydrogen storage
### Hydrogen filled salt caverns

esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=133,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
                   investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
                   economicLifetime=30))


## 5.3. Methane storage
### Methane filled salt caverns

esM.add(fn.Storage(esM=esM, name='Salt caverns (biogas)', commodity='biogas',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=443,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (methane), capacityMax'],
                   investPerCapacity=0.00004, opexPerCapacity=0.00001, interestRate=0.08,
                   economicLifetime=30))


## 5.4 Pumped hydro storage
### Pumped hydro storage

esM.add(fn.Storage(esM=esM, name='Pumped hydro storage', commodity='electricity',
                   chargeEfficiency=0.88, dischargeEfficiency=0.88,
                   hasCapacityVariable=True, selfDischarge=1-(1-0.00375)**(1/(30*24)),
                   chargeRate=0.16, dischargeRate=0.12, capacityFix=data['Pumped hydro storage, capacityFix'],
                   investPerCapacity=0, opexPerCapacity=0.000153))


# 6. Add commodity transmission components to the energy system model
## 6.1. Electricity transmission
### AC cables

esM.add(fn.LinearOptimalPowerFlow(esM=esM, name='AC cables', commodity='electricity',
                                  hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix'],
                                  reactances=data['AC cables, reactances']))

### DC cables

esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                        distances=data['DC cables, distances'],
                        hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))


## 6.2 Methane transmission
### Methane pipeline

esM.add(fn.Transmission(esM=esM, name='Pipelines (biogas)', commodity='biogas',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=False, bigM=300,
                            capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                            investPerCapacity=0.000037, investIfBuilt=0.000314,
                            interestRate=0.08, economicLifetime=40))

## 6.3 Hydrogen transmission
### Hydrogen pipelines

esM.add(fn.Transmission(esM=esM, name='Pipelines (hydrogen)', commodity='hydrogen',
                            distances=data['Pipelines, distances'],
                            hasCapacityVariable=True, hasIsBuiltBinaryVariable=False, bigM=300,
                            locationalEligibility=data['Pipelines, eligibility'],
                            capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                            investPerCapacity=0.000177, investIfBuilt=0.00033,
                            interestRate=0.08, economicLifetime=40))

# 7. Add commodity sinks to the energy system model
## 7.1. Electricity sinks
### Electricity demand

esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                    hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

## 7.2. Hydrogen sinks
### Fuel cell electric vehicle (FCEV) demand

FCEV_penetration=0.5
esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
                    operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))

## 7.3. CO2 sinks
### CO2 exiting the system's boundary

esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                    hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 8. Optimize energy system model

# esM.cluster(numberOfTypicalPeriods=3)

# esM.optimize(timeSeriesAggregation=True, solver = solver)

The distances of a component are set to a normalized value of 1.


 c:\users\s.patil\documents\code\hackthon\fine\FINE\utils.py:671: UserWarning:A declaration of bigM is not necessary if hasIsBuiltBinaryVariable is set to false. The value of bigM will be ignored in the optimization.
 c:\users\s.patil\documents\code\hackthon\fine\FINE\utils.py:671: UserWarning:A declaration of bigM is not necessary if hasIsBuiltBinaryVariable is set to false. The value of bigM will be ignored in the optimization.


In [5]:
esm_dict, component_dict = fn.dictIO.exportToDict(esM)

 c:\users\s.patil\documents\code\hackthon\fine\FINE\utils.py:1050: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\hackthon\fine\FINE\utils.py:1050: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\hackthon\fine\FINE\utils.py:1050: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\hackthon\fine\FINE\utils.py:1050: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\hackthon\fine\FINE\utils.py:1050: UserWarning:Panda

In [6]:
esm_dict


{'locations': {'cluster_0',
  'cluster_1',
  'cluster_2',
  'cluster_3',
  'cluster_4',
  'cluster_5',
  'cluster_6',
  'cluster_7'},
 'commodities': {'CO2', 'biogas', 'electricity', 'hydrogen', 'methane'},
 'commodityUnitsDict': {'electricity': 'GW$_{el}$',
  'methane': 'GW$_{CH_{4},LHV}$',
  'biogas': 'GW$_{biogas,LHV}$',
  'CO2': 'Mio. t$_{CO_2}$/h',
  'hydrogen': 'GW$_{H_{2},LHV}$'},
 'numberOfTimeSteps': 8760,
 'hoursPerTimeStep': 1,
 'costUnit': '1e9 Euro',
 'lengthUnit': 'km',
 'verboseLogLevel': 0,
 'balanceLimit': None,
 'lowerBound': False}

In [7]:
component_dict

{'Source': {'Wind (onshore)': {'name': 'Wind (onshore)',
   'commodity': 'electricity',
   'hasCapacityVariable': True,
   'capacityVariableDomain': 'continuous',
   'capacityPerPlantUnit': 1,
   'hasIsBuiltBinaryVariable': False,
   'bigM': None,
   'operationRateMax':       cluster_0  cluster_1  cluster_2  cluster_3  cluster_4  cluster_5  \
   0      0.569672   0.984958   0.987930   0.851416   0.276660   0.758013   
   1      0.582094   0.972450   0.976257   0.853392   0.275148   0.757377   
   2      0.608804   0.951505   0.936413   0.848352   0.286690   0.754470   
   3      0.637364   0.913621   0.887258   0.833918   0.303492   0.755537   
   4      0.659234   0.874988   0.822736   0.817766   0.324501   0.769687   
   ...         ...        ...        ...        ...        ...        ...   
   8755   0.313734   0.673692   0.757970   0.343864   0.155183   0.320427   
   8756   0.279877   0.642817   0.740666   0.289150   0.119844   0.294879   
   8757   0.270794   0.589719   0.78920

In [8]:
output_df_dict, output_series_dict = fn.xarrayIO.generate_iteration_dicts(component_dict)

In [9]:
output_df_dict

{'operationRateMax': [('Source', 'Wind (onshore)'),
  ('Source', 'Wind (offshore)'),
  ('Source', 'PV'),
  ('Source', 'Biogas purchase')],
 'operationRateFix': [('Source', 'Existing run-of-river plants'),
  ('Sink', 'Electricity demand'),
  ('Sink', 'Hydrogen demand')],
 'commodityCostTimeSeries': [('Source', 'Natural gas purchase'),
  ('Source', 'Biogas purchase')]}

In [10]:
output_series_dict

{'locationalEligibility': [('Source', 'Wind (onshore)'),
  ('Source', 'Wind (offshore)'),
  ('Source', 'PV'),
  ('Source', 'Existing run-of-river plants'),
  ('Source', 'Natural gas purchase'),
  ('Source', 'Biogas purchase'),
  ('Source', 'CO2 from enviroment'),
  ('Sink', 'Electricity demand'),
  ('Sink', 'Hydrogen demand'),
  ('Sink', 'CO2 to enviroment'),
  ('Conversion', 'CCGT plants (methane)'),
  ('Conversion', 'New CCGT plants (biogas)'),
  ('Conversion', 'New CCGT plants (hydrogen)'),
  ('Conversion', 'Electroylzers'),
  ('Conversion', 'rSOEC'),
  ('Conversion', 'rSOFC'),
  ('Storage', 'Li-ion batteries'),
  ('Storage', 'Salt caverns (hydrogen)'),
  ('Storage', 'Salt caverns (biogas)'),
  ('Storage', 'Pumped hydro storage'),
  ('LinearOptimalPowerFlow', 'AC cables'),
  ('Transmission', 'DC cables'),
  ('Transmission', 'Pipelines (biogas)'),
  ('Transmission', 'Pipelines (hydrogen)')],
 'capacityMax': [('Source', 'Wind (onshore)'),
  ('Source', 'Wind (offshore)'),
  ('Source', 

In [12]:
esm_dict, component_dict = fn.dictIO.exportToDict(esM)
output_xarray = fn.xarrayIO.dimensional_data_to_xarray_dataset(esm_dict, component_dict)

In [13]:
output_xarray

<xarray.Dataset>
Dimensions:                       (component: 24, level_1: 8760, space: 8, space_2: 8)
Coordinates:
  * component                     (component) object 'Conversion, CCGT plants...
  * level_1                       (level_1) int64 0 1 2 3 ... 8757 8758 8759
  * space                         (space) object 'cluster_0' ... 'cluster_7'
  * space_2                       (space_2) object 'cluster_0' ... 'cluster_7'
Data variables:
    ts_operationRateMax           (component, level_1, space) float64 nan ......
    ts_operationRateFix           (component, level_1, space) float64 nan ......
    ts_commodityCostTimeSeries    (component, level_1, space) float64 nan ......
    2d_locationalEligibility      (component, space, space_2) float64 nan ......
    2d_capacityMax                (component, space, space_2) float64 nan ......
    2d_investPerCapacity          (component, space, space_2) float64 nan ......
    2d_investIfBuilt              (component, space, space_2) float64 nan ......
    2d_opexPerOperation           (component, space, space_2) float64 nan ......
    2d_opexPerCapacity            (component, space, space_2) float64 nan ......
    2d_opexIfBuilt                (component, space, space_2) float64 nan ......
    2d_QPcostScale                (component, space, space_2) float64 nan ......
    2d_interestRate               (component, space, space_2) float64 nan ......
    2d_economicLifetime           (component, space, space_2) float64 nan ......
    2d_technicalLifetime          (component, space, space_2) float64 nan ......
    2d_capacityFix                (component, space, space_2) float64 nan ......
    2d_reactances                 (component, space, space_2) float64 nan ......
    2d_losses                     (component, space, space_2) float64 nan ......
    2d_distances                  (component, space, space_2) float64 nan ......
    1d_locationalEligibility      (component, space) float64 1.0 1.0 ... nan nan
    1d_capacityMax                (component, space) float64 nan nan ... nan nan
    1d_investPerCapacity          (component, space) float64 0.65 0.65 ... nan
    1d_investIfBuilt              (component, space) float64 0.0 0.0 ... nan nan
    1d_opexPerOperation           (component, space) float64 0.0 0.0 ... nan nan
    1d_commodityCost              (component, space) float64 nan nan ... nan nan
    1d_commodityRevenue           (component, space) float64 nan nan ... nan nan
    1d_opexPerCapacity            (component, space) float64 0.021 0.021 ... nan
    1d_opexIfBuilt                (component, space) float64 0.0 0.0 ... nan nan
    1d_QPcostScale                (component, space) float64 0.0 0.0 ... nan nan
    1d_interestRate               (component, space) float64 0.08 0.08 ... nan
    1d_economicLifetime           (component, space) float64 33.0 33.0 ... nan
    1d_technicalLifetime          (component, space) float64 33.0 33.0 ... nan
    1d_capacityFix                (component, space) float64 nan nan ... nan nan
    1d_opexPerChargeOperation     (component, space) float64 nan nan ... nan nan
    1d_opexPerDischargeOperation  (component, space) float64 nan nan ... nan nan

In [ ]:
esm_dict.get()

In [17]:
esm_dict.pop('locations')

{'cluster_0',
 'cluster_1',
 'cluster_2',
 'cluster_3',
 'cluster_4',
 'cluster_5',
 'cluster_6',
 'cluster_7'}

In [18]:
output_xarray.attrs = esm_dict

In [21]:
xarray_dataset =output_xarray


In [22]:
esm_dict = xarray_dataset.attrs

esm_dict['locations'] = set(str(value) for value in xarray_dataset.space.values)

In [23]:
esm_dict


{'commodities': {'CO2', 'biogas', 'electricity', 'hydrogen', 'methane'},
 'commodityUnitsDict': {'electricity': 'GW$_{el}$',
  'methane': 'GW$_{CH_{4},LHV}$',
  'biogas': 'GW$_{biogas,LHV}$',
  'CO2': 'Mio. t$_{CO_2}$/h',
  'hydrogen': 'GW$_{H_{2},LHV}$'},
 'numberOfTimeSteps': 8760,
 'hoursPerTimeStep': 1,
 'costUnit': '1e9 Euro',
 'lengthUnit': 'km',
 'verboseLogLevel': 0,
 'balanceLimit': None,
 'lowerBound': False,
 'locations': {'cluster_0',
  'cluster_1',
  'cluster_2',
  'cluster_3',
  'cluster_4',
  'cluster_5',
  'cluster_6',
  'cluster_7'}}

In [29]:
xarray_dataset

<xarray.Dataset>
Dimensions:                       (component: 24, level_1: 8760, space: 8, space_2: 8)
Coordinates:
  * component                     (component) object 'Conversion, CCGT plants...
  * level_1                       (level_1) int64 0 1 2 3 ... 8757 8758 8759
  * space                         (space) object 'cluster_0' ... 'cluster_7'
  * space_2                       (space_2) object 'cluster_0' ... 'cluster_7'
Data variables:
    ts_operationRateMax           (component, level_1, space) float64 nan ......
    ts_operationRateFix           (component, level_1, space) float64 nan ......
    ts_commodityCostTimeSeries    (component, level_1, space) float64 nan ......
    2d_locationalEligibility      (component, space, space_2) float64 nan ......
    2d_capacityMax                (component, space, space_2) float64 nan ......
    2d_investPerCapacity          (component, space, space_2) float64 nan ......
    2d_investIfBuilt              (component, space, space_2) float64 nan ......
    2d_opexPerOperation           (component, space, space_2) float64 nan ......
    2d_opexPerCapacity            (component, space, space_2) float64 nan ......
    2d_opexIfBuilt                (component, space, space_2) float64 nan ......
    2d_QPcostScale                (component, space, space_2) float64 nan ......
    2d_interestRate               (component, space, space_2) float64 nan ......
    2d_economicLifetime           (component, space, space_2) float64 nan ......
    2d_technicalLifetime          (component, space, space_2) float64 nan ......
    2d_capacityFix                (component, space, space_2) float64 nan ......
    2d_reactances                 (component, space, space_2) float64 nan ......
    2d_losses                     (component, space, space_2) float64 nan ......
    2d_distances                  (component, space, space_2) float64 nan ......
    1d_locationalEligibility      (component, space) float64 1.0 1.0 ... nan nan
    1d_capacityMax                (component, space) float64 nan nan ... nan nan
    1d_investPerCapacity          (component, space) float64 0.65 0.65 ... nan
    1d_investIfBuilt              (component, space) float64 0.0 0.0 ... nan nan
    1d_opexPerOperation           (component, space) float64 0.0 0.0 ... nan nan
    1d_commodityCost              (component, space) float64 nan nan ... nan nan
    1d_commodityRevenue           (component, space) float64 nan nan ... nan nan
    1d_opexPerCapacity            (component, space) float64 0.021 0.021 ... nan
    1d_opexIfBuilt                (component, space) float64 0.0 0.0 ... nan nan
    1d_QPcostScale                (component, space) float64 0.0 0.0 ... nan nan
    1d_interestRate               (component, space) float64 0.08 0.08 ... nan
    1d_economicLifetime           (component, space) float64 33.0 33.0 ... nan
    1d_technicalLifetime          (component, space) float64 33.0 33.0 ... nan
    1d_capacityFix                (component, space) float64 nan nan ... nan nan
    1d_opexPerChargeOperation     (component, space) float64 nan nan ... nan nan
    1d_opexPerDischargeOperation  (component, space) float64 nan nan ... nan nan
Attributes:
    commodities:         {'electricity', 'hydrogen', 'biogas', 'CO2', 'methane'}
    commodityUnitsDict:  {'electricity': 'GW$_{el}$', 'methane': 'GW$_{CH_{4}...
    numberOfTimeSteps:   8760
    hoursPerTimeStep:    1
    costUnit:            1e9 Euro
    lengthUnit:          km
    verboseLogLevel:     0
    balanceLimit:        None
    lowerBound:          False
    locations:           {'cluster_4', 'cluster_1', 'cluster_3', 'cluster_6',...

<xarray.Dataset>
Dimensions:                       (level_1: 8760, space: 8, space_2: 8)
Coordinates:
    component                     <U33 'Conversion, CCGT plants (methane)'
  * level_1                       (level_1) int64 0 1 2 3 ... 8757 8758 8759
  * space                         (space) object 'cluster_0' ... 'cluster_7'
  * space_2                       (space_2) object 'cluster_0' ... 'cluster_7'
Data variables:
    ts_operationRateMax           (level_1, space) float64 nan nan ... nan nan
    ts_operationRateFix           (level_1, space) float64 nan nan ... nan nan
    ts_commodityCostTimeSeries    (level_1, space) float64 nan nan ... nan nan
    2d_locationalEligibility      (space, space_2) float64 nan nan ... nan nan
    2d_capacityMax                (space, space_2) float64 nan nan ... nan nan
    2d_investPerCapacity          (space, space_2) float64 nan nan ... nan nan
    2d_investIfBuilt              (space, space_2) float64 nan nan ... nan nan
    2d_opexPerOperation           (space, space_2) float64 nan nan ... nan nan
    2d_opexPerCapacity            (space, space_2) float64 nan nan ... nan nan
    2d_opexIfBuilt                (space, space_2) float64 nan nan ... nan nan
    2d_QPcostScale                (space, space_2) float64 nan nan ... nan nan
    2d_interestRate               (space, space_2) float64 nan nan ... nan nan
    2d_economicLifetime           (space, space_2) float64 nan nan ... nan nan
    2d_technicalLifetime          (space, space_2) float64 nan nan ... nan nan
    2d_capacityFix                (space, space_2) float64 nan nan ... nan nan
    2d_reactances                 (space, space_2) float64 nan nan ... nan nan
    2d_losses                     (space, space_2) float64 nan nan ... nan nan
    2d_distances                  (space, space_2) float64 nan nan ... nan nan
    1d_locationalEligibility      (space) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    1d_capacityMax                (space) float64 nan nan nan ... nan nan nan
    1d_investPerCapacity          (space) float64 0.65 0.65 0.65 ... 0.65 0.65
    1d_investIfBuilt              (space) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    1d_opexPerOperation           (space) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    1d_commodityCost              (space) float64 nan nan nan ... nan nan nan
    1d_commodityRevenue           (space) float64 nan nan nan ... nan nan nan
    1d_opexPerCapacity            (space) float64 0.021 0.021 ... 0.021 0.021
    1d_opexIfBuilt                (space) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    1d_QPcostScale                (space) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    1d_interestRate               (space) float64 0.08 0.08 0.08 ... 0.08 0.08
    1d_economicLifetime           (space) float64 33.0 33.0 33.0 ... 33.0 33.0
    1d_technicalLifetime          (space) float64 33.0 33.0 33.0 ... 33.0 33.0
    1d_capacityFix                (space) float64 nan nan nan ... nan nan nan
    1d_opexPerChargeOperation     (space) float64 nan nan nan ... nan nan nan
    1d_opexPerDischargeOperation  (space) float64 nan nan nan ... nan nan nan
Attributes:
    commodities:         {'electricity', 'hydrogen', 'biogas', 'CO2', 'methane'}
    commodityUnitsDict:  {'electricity': 'GW$_{el}$', 'methane': 'GW$_{CH_{4}...
    numberOfTimeSteps:   8760
    hoursPerTimeStep:    1
    costUnit:            1e9 Euro
    lengthUnit:          km
    verboseLogLevel:     0
    balanceLimit:        None
    lowerBound:          False
    locations:           {'cluster_4', 'cluster_1', 'cluster_3', 'cluster_6',...

In [47]:
component_dict = {}
for component in xarray_dataset.component.values:
    sub_xarray = xarray_dataset.sel(component=component)
    
    for variable in sub_xarray.data_vars:
            
        if not xr.ufuncs.isnan(sub_xarray[variable].values).all():
            print('no na')
            
            # set all regional time series (regions, time)
            if variable[:3]== 'ts_':
                
                print(f'{variable} is chosen')
                df = sub_xarray[variable].drop("component").to_dataframe().unstack(level=2)

                if len(df.columns) > 1:
                    df.columns = df.columns.droplevel(0)
                
                class_component_list = component.split(', ')
                
                if class_component_list[0] not in component_dict.keys():
                    component_dict.update({class_component_list[0]: {}})
                if class_component_list[1] not in component_dict.get(class_component_list[0]).keys():
                    component_dict.get(class_component_list[0]).update(class_component_list[1]: {})
                if [variable[3:]] not in component_dict.get(class_component_list[0]).get(class_component_list[1]).keys():
                    component_dict.get(class_component_list[0]).get(class_component_list[1]).update([variable[3:]]: df.sort_index())
                    
                
            # set all 2d data (regions, regions)
            elif variable[:3]== '2d_':
                print(f'{variable} is chosen')
                series = sub_xarray[variable].drop("component").to_dataframe().stack(level=0)

                series.index = series.index.droplevel(level=2).map('_'.join)
                
                class_component_list = component.split(', ')
                component_dict[class_component_list[0]][class_component_list[1]][variable[3:]] = series.sort_index()
            
            # set all 1d data (regions)
            elif variable[:3]== '1d_':
                print(f'{variable} is chosen')
                series = sub_xarray[variable].drop("component").to_dataframe().unstack(level=0)
                series.index = series.index.droplevel(level=0)
                 
                class_component_list = component.split(', ')
                component_dict[class_component_list[0]][class_component_list[1]][variable[3:]] = series.sort_index()
            
print(component_dict)        
    
    
        
    
#     
    
#     print(comp)

no na
1d_locationalEligibility is chosen


KeyError: 'Conversion'